# Geocodificar usuarios [Base completa]

In [1]:
import pandas as pd
import get_centroides
from get_centroides import getFromDict
import sqlalchemy
import warnings
import pickle

In [2]:
df = pd.read_csv('./HI_usuarios_completos.csv',  encoding='latin-1') # Datos institucion
df.head()

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part
0,2014,AR,CF,CABA,1416
1,2014,AR,CF,CABA,1416
2,2014,AR,BA,HURLINGHAM,1686
3,2014,AR,BA,ZARATE,2800
4,2014,AR,CF,CABA,1425


In [3]:
df.dtypes

anio_ing           int64
nacionalidad      object
provincia_part    object
loc_part          object
cp_part           object
dtype: object

In [4]:
df.nacionalidad.unique() # Algunos NaN y fechas en el campo nacionalidad.

array(['AR', 'CA', 'UY', 'PY', 'IT', 'US', 'AU', 'BR', 'JP', 'BO', 'CN',
       'ES', 'CO', 'VE', 'KR', 'EC', 'PE', 'CL', 'FR', 'GB', 'HL', 'PT',
       'CU', 'SY', 'KP', 'PA', 'CR', 'YU', 'HN', 'PL', 'UA', 'DE', 'MX',
       'GT', 'IL', 'HT', 'AO', 'RU', '30/06/2015', 'IO', 'CH', 'HR', 'AT',
       'LT', 'HU', 'TH', 'ET', '05/08/1987', 'DO', 'IN', 'TW', 'MK', 'SV',
       'NZ', 'GR', 'AM', 'DM', 'SK', 'PH', 'SE', 'BG', 'VN', 'DK', 'SI',
       'NI', 'IR', 'FI', 'IE', 'KZ', 'TR', 'LI', nan, 'BE', 'LB', 'BY',
       'IS', 'ZA', 'NG', 'MZ', 'BD', 'PS'], dtype=object)

In [5]:
df.provincia_part.unique() # Parece ser que provincia es un campo estructurado

array(['CF', 'BA', nan, 'LP', 'MZ', 'SL', 'CA', 'SC', 'CH', 'RN', 'TF',
       'SF', 'ER', 'MI', 'CB', 'TU', 'NE', 'CO', 'JU', 'SJ', 'SA', 'FO',
       'SE', 'CT'], dtype=object)

In [6]:
df.nacionalidad.str.contains('/').sum() # Tengo el mismo problema que antes

2

In [7]:
df.isna().sum() 
# 206 usuarios sin  nacionalidad
# 4820 sin provincia reportada
# 1 solo sin CP    

anio_ing             0
nacionalidad       206
provincia_part    4820
loc_part           263
cp_part              1
dtype: int64

Pareceria que todo es muy similar al dataset mas pequeño con el que trabajamos antes. Exploraremos algunas aspectos mas granulares (ej por nacionalidad) parar ver si en los casos mas 'anomalos' hay algun conflicto.

In [8]:
exp_gb = df.groupby(['anio_ing','nacionalidad','provincia_part']).agg('count').reset_index()

In [9]:
exp_gb.isna().sum() # Los datos de gente de otra nacionalidad parecerian estar todos completos

anio_ing          0
nacionalidad      0
provincia_part    0
loc_part          0
cp_part           0
dtype: int64

In [10]:
exp_gb[exp_gb.nacionalidad != 'AR'].provincia_part.unique() 
# Parece que todos las personas que reportan otra nacionalidad igualmente seleccionan una provincia
# en el territorio argentino

array(['BA', 'CF', 'SA', 'RN', 'CO', 'MI', 'CA'], dtype=object)

## Preprocessing

Vamos a remover los dos sujetos que presentan fechas en las nacionalidades. 

In [11]:
# Vamos a asumir que los campos con valor None en nacionalidad se deben a errores de los usuarios, pasaran a ser `desconocido`.
# Los campos que tengan fechas en la nacionalidad seran removidos.
df.nacionalidad[df.nacionalidad.isna()] = 'DESCONOCIDO'

C:\Users\joaqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_clean = df.loc[df.nacionalidad.str.contains('/') == False].reset_index(drop = True).copy()

In [13]:
df_clean.shape[0] - df.shape[0] # Eliminamos dos usuarios de 80k

-2

Para este caso asignare un `user_id` simulando un dataset real. 

In [14]:
df_clean['user_id'] = list(map(lambda x: 'ID_' + str(x), df_clean.index.to_list()))

In [15]:
# Poner loc-part todo en mayuscula
df_clean.loc_part = df_clean.loc_part.apply(lambda x: x.upper() if isinstance(x,str) else None)  

## Acceder a la base de datos 

In [16]:
warnings.simplefilter("ignore", category=sqlalchemy.exc.SAWarning) # ignor sqlalchemy warnings
engine = sqlalchemy.create_engine('sqlite:///SQL/CPA.db')
con = engine.connect()
cpa_prov = pd.read_sql('provincia', con=con) # provincia por codigo postal
cpa_local = pd.read_sql('localidad', con=con) # localidad por codigo postal
#Se debe remover este campo del dataframe antes de realizar el join.
cpa_local.drop(cpa_local[cpa_local.codigopostal == 0].index).reset_index(drop = True, inplace=True)
cpa_capital =  pd.read_sql('comunas_capital', con=con) # localidades y comuna de capital por CP

In [17]:
cpa_provincia_completo = cpa_local.merge(cpa_prov, left_on='provincia_id', right_on='p_id',
                                         suffixes=['_localidad','_provincia']).drop(columns = 'p_id')
CPA_ARGENTINA =  cpa_provincia_completo.merge(cpa_capital, on= 'provincia_id' ,
                                              how='left' , suffixes = ('_x',''))  
# El dataframe mas completo hasta el momento de CPA y localidad
# Unir los dos codigos postales de las distintas tablas
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.fillna(CPA_ARGENTINA.codigopostal_x)
CPA_ARGENTINA.drop(columns='codigopostal_x', inplace= True) # eliminar la columna CP repetida
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.astype(int) # Transformar a integer el CP

In [18]:
set_provincia = set(cpa_provincia_completo.codigopostal)
set_provincia.remove(0) # Nuevamente removemos el 0 del codigo postal de provincia
set_capital = set(cpa_capital.codigopostal)
set_capital.intersection(set_provincia) # No hay CP en capital que se repitan en las otras provincias

set()

In [19]:
## Acceder a los datos de la API
departamentos = pd.read_csv('https://apis.datos.gob.ar/georef/api/departamentos.csv')
localidades = pd.read_csv('https://apis.datos.gob.ar/georef/api/localidades.csv')
municipios = pd.read_csv('https://apis.datos.gob.ar/georef/api/municipios.csv')
provincias = pd.read_csv('https://apis.datos.gob.ar/georef/api/provincias.csv')

## Populando nueva tabla con datos de geolocalizacion
### Asignar localidad, departamento(comuna CP) y provincia usando  solo CP

In [20]:
# Listado de CP que no son unicos
provincia_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_provincia'].nunique()
cp_prov_no_unicos = set(provincia_por_cp.loc[provincia_por_cp > 1].index)

localidad_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_localidad'].nunique()
cp_localidad_no_unicos = set(localidad_por_cp.loc[localidad_por_cp > 1].index) 

capital_barrio_por_cp = cpa_capital.groupby('codigopostal')['barrio'].nunique()
capital_cp_barrio_no_unicos = set(capital_barrio_por_cp.loc[capital_barrio_por_cp > 1].index)

capital_comuna_por_cp = cpa_capital.groupby('codigopostal')['comuna'].nunique()
capital_cp_comuna_no_unicos = set(capital_comuna_por_cp.loc[capital_comuna_por_cp > 1].index)

In [21]:
# Funciones utiles
def infer_provincia(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    if int(cp) in set_capital:
        return 'CIUDAD AUTONOMA DE BUENOS AIRES'
    if int(cp) in set_provincia:
        if  int(cp) in cp_prov_no_unicos:
            return  'NO UNICA PROVINCIA'
        else:
            prov = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == int(cp)].nombre_provincia.unique()
            return prov[0].upper()
    if (int(cp) not in set_provincia and int(cp) not in set_capital):
        return 'CP INDEFINIDO'
    if cp.isna() or int(cp) == 0:
        return 'CP INDEFINIDO'

def infer_localidad(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
                return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_capital.loc[cpa_capital.codigopostal == cp].barrio.unique()[0]
            return localidad.upper()
    if cp in set_provincia:
        if cp in cp_localidad_no_unicos:
            return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == cp].nombre_localidad.unique()[0]
            return localidad.upper()
    if cp not in set_provincia and cp not in set_capital:
        return 'CP INDEFINIDO'
           
def infer_departamento(cp):
    try:
        int(cp)
    except:
        return 'CP INDEFINIDO'
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
            return 'NO UNICO DEPARTAMENTO'
        else:
            departamento = cpa_capital.loc[cpa_capital.codigopostal == cp].comuna.unique()
            return f'COMUNA {departamento[0]}'

In [22]:
augmented_df = df_clean.copy() # este es el dataframe que vamos a popular
#Agregamos columnas
columnas =  ['provincia_por_cp' ,'localidad_por_cp' ,'departamento_por_cp' ,'localidad_por_API', 'localidad_id_API', 'municipio_por_API',
             'municipio_id_API', 'departamento_por_API', 'departamento_id_API', 'provincia_por_API', 'provincia_id_API','centroide_lat_localidad',
             'centroide_lon_localidad', 'centroide_lat_municipio','centroide_lon_municipio','centroide_lat_departamento','centroide_lon_departamento',
             'centroide_lat_provincia','centroide_lon_provincia']
for i in columnas:
    augmented_df[i] = None

In [23]:
# Provincia
augmented_df['provincia_por_cp'] = augmented_df.cp_part.apply(infer_provincia)
# Localidad
augmented_df['localidad_por_cp'] = augmented_df.cp_part.apply(infer_localidad)
# Departamento 'Comuna'
augmented_df['departamento_por_cp'] = augmented_df.cp_part.apply(infer_departamento)

In [24]:
 augmented_df.sample(10)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,...,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
53024,2017,AR,CF,CABA,1234,ID_53024,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
45382,2017,AR,BA,CABA,1168,ID_45382,CIUDAD AUTONOMA DE BUENOS AIRES,MONTSERRAT,COMUNA 1,None,...,None,None,None,None,None,None,None,None,None,None
60909,2018,AR,CF,CABA,1181,ID_60909,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
51545,2017,AR,CF,CABA,1440,ID_51545,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
4894,2014,AR,CF,CABA,1228,ID_4894,CIUDAD AUTONOMA DE BUENOS AIRES,BOEDO,COMUNA 5,None,...,None,None,None,None,None,None,None,None,None,None
76516,2019,AR,CF,CABA,1439,ID_76516,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
73248,2018,AR,CF,CABA,1405,ID_73248,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
20271,2015,AR,CF,CABA,1217,ID_20271,CIUDAD AUTONOMA DE BUENOS AIRES,SAN TELMO,COMUNA 1,None,...,None,None,None,None,None,None,None,None,None,None
7176,2014,AR,CF,CABA,1110,ID_7176,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,None,...,None,None,None,None,None,None,None,None,None,None
45618,2017,AR,CF,CDAD AUTONOMA BS AS,8400,ID_45618,NO UNICA PROVINCIA,NO UNICA LOCALIDAD,None,None,...,None,None,None,None,None,None,None,None,None,None


### Conseguir datos de los que tienen provincia y localidad provista por codigo postal

In [25]:
# Seleccionamos solo las prov con loc
gby_prov_loc = augmented_df.groupby(['provincia_por_cp','localidad_por_cp']).count().reset_index(level=['provincia_por_cp', 'localidad_por_cp']).copy()
gby_prov_loc = gby_prov_loc.loc[(gby_prov_loc.localidad_por_cp != 'NO UNICA LOCALIDAD') & (gby_prov_loc.localidad_por_cp != 'CP INDEFINIDO')]

In [26]:
gby_prov_loc.drop(columns = ['anio_ing','user_id','nacionalidad','provincia_part','cp_part','loc_part','departamento_por_cp'], inplace= True)
gby_prov_loc.loc[:,'localidad_por_API' : 'centroide_lon_provincia'] = None # inicializo todos los campos a zero
print(gby_prov_loc.shape)
gby_prov_loc.head(4) # Tenemos 105 combinacinos unicas no identificadas

(105, 18)


,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,BUENOS AIRES,9 DE ABRIL,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,BUENOS AIRES,AEROPUERTO EZEIZA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,BUENOS AIRES,ALEJANDRO KORN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,BUENOS AIRES,BANFIELD,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [27]:
# Crea una lista de tuplas para realizar el post. Para mas info consultar get_centroides.py
prov_loc = list(gby_prov_loc.apply(lambda x: (x.provincia_por_cp, x.localidad_por_cp), axis=1))
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [28]:
#funcion para rellenar los df
def rellenar_df(df,datos):
    df['provincia_por_API'] = [getFromDict(i,'provincia','nombre') for i in datos]
    df['provincia_id_API'] = [getFromDict(i,'provincia','id') for i in datos]
    df['departamento_por_API'] = [getFromDict(i,'departamento','nombre') for i in datos]
    df['departamento_id_API'] = [getFromDict(i,'departamento','id') for i in datos]
    df['municipio_por_API'] = [getFromDict(i,'municipio','nombre') for i in datos]
    df['municipio_id_API'] = [getFromDict(i,'municipio','id') for i in datos]
    df['localidad_por_API'] = [getFromDict(i,'nombre') for i in datos]
    df['localidad_id_API'] = [getFromDict(i,'id') for i in datos]
    df['centroide_lat_localidad'] = [getFromDict(i,'centroide','lat') for i in datos]
    df['centroide_lon_localidad'] = [getFromDict(i,'centroide','lon') for i in datos]
    cols_s = ['municipio_por_API', 'departamento_por_API', 'provincia_por_API']
    cols_i = ['departamento_id_API','municipio_id_API','provincia_id_API']
    # To upper
    for i in cols_s:
        df[i] = df[i].apply(lambda x: x.upper() if isinstance(x,str) else 'NO HUBO REQUEST')
    for i in cols_i:
        df[i] = df[i].apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and pd.notna(x) else x)) 
    return df

In [29]:
gby_prov_loc = rellenar_df(gby_prov_loc,datos)
gby_prov_loc.sample(5)

,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
49,BUENOS AIRES,PASO DEL REY,PASO DEL REY,06560010005,MORENO,60560,MORENO,6560,BUENOS AIRES,6,-34.6354,-58.7503,None,None,None,None,None,None
56,BUENOS AIRES,RANELAGH,RANELAGH,06091010009,BERAZATEGUI,60091,BERAZATEGUI,6091,BUENOS AIRES,6,-34.7941,-58.199,None,None,None,None,None,None
26,BUENOS AIRES,INGENIERO ADOLFO SOURDEAUX,INGENIERO ADOLFO SOURDEAUX,06515010003,MALVINAS ARGENTINAS,60515,MALVINAS ARGENTINAS,6515,BUENOS AIRES,6,-34.5004,-58.6618,None,None,None,None,None,None
64,BUENOS AIRES,SANTA TERESITA,SANTA TERESITA,06420040002,LA COSTA,60420,LA COSTA,6420,BUENOS AIRES,6,-36.5409,-56.7059,None,None,None,None,None,None
8,BUENOS AIRES,BOSQUES,BOSQUES,06274010001,FLORENCIO VARELA,60274,FLORENCIO VARELA,6274,BUENOS AIRES,6,-34.8276,-58.2229,None,None,None,None,None,None


In [30]:
gby_prov_loc['copy_index'] = gby_prov_loc.index

#Asignar centroide de municipios
gby_prov_loc.municipio_id_API = gby_prov_loc.municipio_id_API.apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and pd.notna(x) else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_municipio = gby_prov_loc.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_municipio = gby_prov_loc.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

#Asignar centroide de departamentos
gby_prov_loc.departamento_id_API = gby_prov_loc.departamento_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

#Asignar centroide de provincias
gby_prov_loc.provincia_id_API = gby_prov_loc.provincia_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

In [31]:
gby_prov_loc.sample(5)

,provincia_por_cp,localidad_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index
101,CIUDAD AUTONOMA DE BUENOS AIRES,RECOLETA,RECOLETA,02014010001,NO HUBO REQUEST,None,COMUNA 2,2014,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.5859,-58.395,NaN,NaN,-34.585765,-58.394892,-34.614493,-58.445856,101
2,BUENOS AIRES,ALEJANDRO KORN,ALEJANDRO KORN,06778020001,SAN VICENTE,60778,SAN VICENTE,6778,BUENOS AIRES,6,-35.0135,-58.3865,-35.070460,-58.431922,-35.070460,-58.431922,-36.676942,-60.558832,2
7,BUENOS AIRES,BENAVIDEZ,BENAVIDEZ,06805010001,TIGRE,60805,TIGRE,6805,BUENOS AIRES,6,-34.4104,-58.6888,-34.381589,-58.599454,-34.381589,-58.599454,-36.676942,-60.558832,7
57,BUENOS AIRES,REMEDIOS DE ESCALADA,REMEDIOS DE ESCALADA,06840010011,TRES DE FEBRERO,60840,TRES DE FEBRERO,6840,BUENOS AIRES,6,-34.5706,-58.6201,-34.595980,-58.579093,-34.595980,-58.579093,-36.676942,-60.558832,57
31,BUENOS AIRES,JOSE CLEMENTE PAZ,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,31


#### Agregar los datos a la tabla aumentada

In [32]:
pd.set_option('display.max_columns', 50)  # Para ver todas las filas
# Vamos a hacer un Join con los valores que tengan la misma provincia y localidad por CP
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales
gby_prov_loc.drop(columns= 'copy_index', inplace= True)

augmented_df = augmented_df.merge(gby_prov_loc , on = ['provincia_por_cp','localidad_por_cp'],
                                  how= 'left', suffixes= ['_del','']).set_index(augmented_df['copy_index']).copy()
                                                                                 
augmented_df = augmented_df.drop(columns = augmented_df.filter(regex = '._del').columns)
augmented_df.drop(columns = ['copy_index'], inplace = True)

In [33]:
augmented_df.sample(5)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
47724,2017,AR,CF,CABA,1425,ID_47724,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11944,2014,AR,CF,CABA,1430,ID_11944,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21814,2015,AR,CF,CABA,1405,ID_21814,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60283,2014,AR,NaN,REMEDIOS DE ESCALADA,0,ID_60283,CP INDEFINIDO,CP INDEFINIDO,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76580,2019,AR,CF,CABA,1201,ID_76580,CIUDAD AUTONOMA DE BUENOS AIRES,BALVANERA,COMUNA 3,BALVANERA,02021010001,NO HUBO REQUEST,None,COMUNA 3,2021,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6091,-58.4031,NaN,NaN,-34.613843,-58.402651,-34.614493,-58.445856


### Conseguir datos de los que tienen solo provincia por codigo postal

In [34]:
gby_solo_provincia = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida pero si me dio provincia
gby_solo_provincia = gby_solo_provincia.loc[(gby_solo_provincia.provincia_por_cp != 'CP INDEFINIDO') &
                                            (gby_solo_provincia.provincia_por_cp != 'NO UNICA PROVINCIA') &
                                            (gby_solo_provincia.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_provincia.shape)
gby_solo_provincia.head(5)

(534, 25)


,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,VENEZUELA Y PERU,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,-1,BUENOS AIRES,NO UNICA LOCALIDAD,22,22,22,22,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,-1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,-1,SAN JUAN,NO UNICA LOCALIDAD,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,-1,SAN LUIS,NO UNICA LOCALIDAD,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_provincia.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  
#En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [36]:
len(datos)

534

In [37]:
# Rellena la tabla con los datos
gby_solo_provincia = rellenar_df(gby_solo_provincia,datos)
gby_solo_provincia.sample(5)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
218,CARAPACHAI,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,CARAPACHAY,06861010001,VICENTE LÓPEZ,60861,VICENTE LÓPEZ,6861,BUENOS AIRES,6,-34.5259,-58.5358,0,0,0,0,0,0
1255,SUIPACHA,BUENOS AIRES,NO UNICA LOCALIDAD,6,6,6,6,6,0,SUIPACHA,06784020000,SUIPACHA,60784,SUIPACHA,6784,BUENOS AIRES,6,-34.7713,-59.6879,0,0,0,0,0,0
462,EL CALAFATE,SANTA CRUZ,NO UNICA LOCALIDAD,1,1,1,1,1,0,EL CALAFATE,78028010000,NO HUBO REQUEST,None,LAGO ARGENTINO,78028,SANTA CRUZ,78,-50.3373,-72.262,0,0,0,0,0,0
253,CASTELAR,BUENOS AIRES,NO UNICA LOCALIDAD,6,6,5,6,6,0,CASTELAR,06568010001,MORÓN,60568,MORÓN,6568,BUENOS AIRES,6,-34.6644,-58.6456,0,0,0,0,0,0
1306,TRES ARROYOS,BUENOS AIRES,NO UNICA LOCALIDAD,3,3,3,3,3,0,TRES ARROYOS,06833100000,TRES ARROYOS,60833,TRES ARROYOS,6833,BUENOS AIRES,6,-38.3772,-60.2756,0,0,0,0,0,0


In [38]:
gby_solo_provincia['copy_index_2'] = gby_solo_provincia.index

# Asignar centroide de municipios
gby_solo_provincia.centroide_lat_municipio = gby_solo_provincia.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_municipio = gby_solo_provincia.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']
# Asignar centroide de departamentos
gby_solo_provincia.centroide_lon_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']
gby_solo_provincia.centroide_lat_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
# Asignar centroide de provincias
gby_solo_provincia.centroide_lat_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']

In [39]:
gby_solo_provincia.sample(5) 

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index_2
870,MANZANARES,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,1,0,MANZANARES,06638040005,PILAR,60638,PILAR,6638,BUENOS AIRES,6,-34.4526,-59.0137,-34.448199,-58.903441,-34.448199,-58.903441,-36.676942,-60.558832,870
1112,ROJAS,BUENOS AIRES,NO UNICA LOCALIDAD,2,2,2,2,2,0,ROJAS,06686060000,ROJAS,60686,ROJAS,6686,BUENOS AIRES,6,-34.1962,-60.7333,-34.192604,-60.787971,-34.192604,-60.787971,-36.676942,-60.558832,1112
1346,VICENTE LOPEZ,BUENOS AIRES,NO UNICA LOCALIDAD,85,85,81,85,85,0,VICENTE LOPEZ,06861010007,VICENTE LÓPEZ,60861,VICENTE LÓPEZ,6861,BUENOS AIRES,6,-34.5255,-58.4736,-34.526513,-58.504470,-34.526513,-58.504470,-36.676942,-60.558832,1346
1222,SAN PEDRO,BUENOS AIRES,NO UNICA LOCALIDAD,13,13,13,13,13,0,SAN PEDRO,06770050000,SAN PEDRO,60770,SAN PEDRO,6770,BUENOS AIRES,6,-33.6791,-59.6669,-33.781943,-59.782592,-33.781943,-59.782592,-36.676942,-60.558832,1222
36,9 DE JULIO,BUENOS AIRES,NO UNICA LOCALIDAD,3,3,3,3,3,0,9 DE JULIO,06588100000,9 DE JULIO,60588,9 DE JULIO,6588,BUENOS AIRES,6,-35.4447,-60.8843,-35.481191,-60.975116,-35.481191,-60.975116,-36.676942,-60.558832,36


###  Conseguir datos de los que no obtuvimos ni provincia ni localidad


In [40]:
gby_solo_loc_part = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida, ni provincia o tambien si el CP es indefinido
gby_solo_loc_part = gby_solo_loc_part.loc[(gby_solo_loc_part.provincia_por_cp == 'CP INDEFINIDO') |
                                            (gby_solo_loc_part.provincia_por_cp == 'NO UNICA PROVINCIA') &
                                            (gby_solo_loc_part.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_loc_part.shape)
gby_solo_loc_part.sample(5) #221 rows unicas

(221, 25)


,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
729,LA MANUELA,CP INDEFINIDO,CP INDEFINIDO,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
553,GENERAL PACHECO,CP INDEFINIDO,CP INDEFINIDO,4,4,3,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
589,GONZALEZ CATAN,CP INDEFINIDO,CP INDEFINIDO,41,41,14,41,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1042,PRESIDENTE DERQUI,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
478,EL TALAR,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_loc_part.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  # no necesito saber que no se sabe la localidad
# En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request
datos = get_centroides.post_centroides('localidades', prov_loc, prov = False) # Consulta get_centroides.py, no uso la provincia en este caso

In [42]:
len(datos)

221

In [43]:
# Rellena la tabla con los datos
gby_solo_loc_part = rellenar_df(gby_solo_loc_part,datos)
gby_solo_loc_part.sample(5)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
1430,VILLA LUZURIAGA,CP INDEFINIDO,CP INDEFINIDO,447,447,439,447,447,0,VILLA LUZURIAGA,06427010014,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6731,-58.5939,0,0,0,0,0,0
671,ISIDRO CASANOVA,CP INDEFINIDO,CP INDEFINIDO,40,40,14,40,40,0,ISIDRO CASANOVA,06427010005,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.7184,-58.5778,0,0,0,0,0,0
713,JOSÉ MARMOL,CP INDEFINIDO,CP INDEFINIDO,2,2,2,2,2,0,JOSE MARMOL,06028010006,ALMIRANTE BROWN,60028,ALMIRANTE BROWN,6028,BUENOS AIRES,6,-34.7851,-58.3681,0,0,0,0,0,0
189,CABA,NO UNICA PROVINCIA,NO UNICA LOCALIDAD,1,1,1,1,1,0,CABAL,82063040000,CABAL,822966,LA CAPITAL,82063,SANTA FE,82,-31.104,-60.7272,0,0,0,0,0,0
1464,VILLA SANTOS TESEI,CP INDEFINIDO,CP INDEFINIDO,4,4,4,4,4,0,VILLA SANTOS TESEI,06408010002,HURLINGHAM,60408,HURLINGHAM,6408,BUENOS AIRES,6,-34.6124,-58.6538,0,0,0,0,0,0


In [44]:
gby_solo_loc_part['copy_index_2'] = gby_solo_loc_part.index
# Asignar centroide de municipios
gby_solo_loc_part.centroide_lat_municipio = gby_solo_loc_part.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_municipio = gby_solo_loc_part.merge(municipios, left_on= 'municipio_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
# Asignar centroide de departamentos
gby_solo_loc_part.centroide_lon_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
gby_solo_loc_part.centroide_lat_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
# Asignar centroide de provincias
gby_solo_loc_part.centroide_lat_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']
gby_solo_loc_part.sample(10)

,loc_part,provincia_por_cp,localidad_por_cp,anio_ing,nacionalidad,provincia_part,cp_part,user_id,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,copy_index_2
913,MIRAMAR,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,MIRAMAR,06280040000,GENERAL ALVARADO,60280,GENERAL ALVARADO,6280,BUENOS AIRES,6,-38.2707,-57.8405,-38.202713,-58.072249,-38.202713,-58.072249,-36.676942,-60.558832,913
1139,SAN ANDRES,CP INDEFINIDO,CP INDEFINIDO,2,2,1,2,2,0,SAN ANDRES,90014130000,SAN ANDRES,908154,CRUZ ALTA,90014,TUCUMÁN,90,-26.8885,-65.1963,-26.919873,-65.182001,-26.918244,-64.975734,-26.947800,-65.364758,1139
1087,RAMOS MEJIA,CP INDEFINIDO,CP INDEFINIDO,39,39,29,39,39,0,RAMOS MEJIA,06427010009,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6523,-58.5592,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832,1087
1196,SAN LAUREANO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,1,1,0,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NaN,NaN,1196
1418,VILLA KRAUSE,CP INDEFINIDO,CP INDEFINIDO,1,1,0,1,1,0,VILLA KRAUSE,70077010003,RAWSON,700077,RAWSON,70077,SAN JUAN,70,-31.5837,-68.5411,-31.686586,-68.467563,-31.686586,-68.467563,-30.865368,-68.889491,1418
113,BECCAR,CP INDEFINIDO,CP INDEFINIDO,1,1,0,1,1,0,BECCAR,06756010002,SAN ISIDRO,60756,SAN ISIDRO,6756,BUENOS AIRES,6,-34.4674,-58.54,-34.486950,-58.537265,-34.486950,-58.537265,-36.676942,-60.558832,113
1452,VILLA RAFFO,CP INDEFINIDO,CP INDEFINIDO,3,3,3,3,3,0,VILLA RAFFO,06840010015,TRES DE FEBRERO,60840,TRES DE FEBRERO,6840,BUENOS AIRES,6,-34.6083,-58.532,-34.595980,-58.579093,-34.595980,-58.579093,-36.676942,-60.558832,1452
525,FLORIDA,CP INDEFINIDO,CP INDEFINIDO,13,13,11,13,13,0,FLORIDA,06861010002,VICENTE LÓPEZ,60861,VICENTE LÓPEZ,6861,BUENOS AIRES,6,-34.5327,-58.4911,-34.526513,-58.504470,-34.526513,-58.504470,-36.676942,-60.558832,525
844,LOS POLVORINES,CP INDEFINIDO,CP INDEFINIDO,1,1,0,1,1,0,LOS POLVORINES,06515010005,MALVINAS ARGENTINAS,60515,MALVINAS ARGENTINAS,6515,BUENOS AIRES,6,-34.5079,-58.7013,-34.487304,-58.712133,-34.487304,-58.712133,-36.676942,-60.558832,844
1277,TAPIALES,CP INDEFINIDO,CP INDEFINIDO,8,8,8,8,8,0,TAPIALES,06427010011,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.7108,-58.495,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832,1277


In [45]:
gby_solo_loc_part[gby_solo_loc_part['localidad_por_API'] == 'NO HUBO REQUEST'].shape #34 request no devolvieron resultados

(34, 26)

## Agregar todos los datos a la tabla

In [46]:
aux_df = gby_solo_loc_part.append(gby_solo_provincia, sort = False) #append all tables in one df. Except prov_loc 

In [47]:
# Vamos a hacer un mege con los valores que tengan la misma provincia y localidad por CP y loc_part
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales

augmented_df2 = augmented_df.merge(aux_df ,
                                   on = ['loc_part','provincia_por_cp','localidad_por_cp'],
                                   how= 'left',
                                   suffixes= ['','_added']).set_index(augmented_df['copy_index']).copy()


In [48]:
def assign_value(row, column = None):
    '''
    Simple function that takes a row and a column and concatenate two columns after permorming a merge
    In this case the right table with the same column name was added a `_added` termination
    '''
    if pd.isna(row[column]):
        return row[column + '_added']
    else:
        return row[column]

In [49]:
col = ['provincia_por_API','localidad_por_API','departamento_por_API','municipio_por_API',
        'centroide_lat_localidad','centroide_lon_localidad','centroide_lat_departamento',
        'centroide_lon_departamento','centroide_lat_municipio','centroide_lon_municipio','centroide_lat_provincia',
        'centroide_lon_provincia','provincia_id_API','departamento_id_API','municipio_id_API',]
for c in col:
    augmented_df2[c] = augmented_df2.apply(assign_value, column = c, axis = 1)

In [50]:
augmented_df2.drop(columns = augmented_df2.filter(regex = '._added').columns, inplace=True) #drop _added columns
augmented_df2.drop(columns = ['copy_index', 'copy_index_2'], inplace = True) #drop las columnas con indices

In [51]:
augmented_df2.sample(10)

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
62232,2018,AR,CF,CABA,1414,ID_62232,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
18394,2015,AR,CF,CABA,1426,ID_18394,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
12347,2014,AR,CF,CABA,1212,ID_12347,CIUDAD AUTONOMA DE BUENOS AIRES,BALVANERA,COMUNA 3,BALVANERA,02021010001,NO HUBO REQUEST,NaN,COMUNA 3,2021,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6091,-58.4031,NaN,NaN,-34.613843,-58.402651,-34.614493,-58.445856
76542,2019,AR,BA,FLORIDA,1602,ID_76542,BUENOS AIRES,NO UNICA LOCALIDAD,None,FLORIDA,NaN,VICENTE LÓPEZ,60861,VICENTE LÓPEZ,6861,BUENOS AIRES,6,-34.5327,-58.4911,-34.526513,-58.504470,-34.526513,-58.504470,-36.676942,-60.558832
87117,2019,AR,BA,AVELLANEDA,1870,ID_87117,BUENOS AIRES,NO UNICA LOCALIDAD,None,AVELLANEDA,NaN,AVELLANEDA,60035,AVELLANEDA,6035,BUENOS AIRES,6,-34.6627,-58.3657,-34.678206,-58.341117,-34.678206,-58.341117,-36.676942,-60.558832
1889,2014,AR,CF,CABA,1425,ID_1889,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
74661,2018,AR,BA,LOMAS DEL MIRADOR,1752,ID_74661,BUENOS AIRES,NO UNICA LOCALIDAD,None,LOMAS DEL MIRADOR,NaN,LA MATANZA,60427,LA MATANZA,6427,BUENOS AIRES,6,-34.6657,-58.5332,-34.770165,-58.625449,-34.770165,-58.625449,-36.676942,-60.558832
82300,2019,AR,CF,CABA,1424,ID_82300,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
82104,2019,AR,BA,CABA,1406,ID_82104,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856


## EDA

In [52]:
augmented_df2[augmented_df2['localidad_por_API'] == 'NO HUBO REQUEST'].shape

(6877, 25)

In [53]:
augmented_df2.shape

(87402, 25)

In [54]:
augmented_df2[augmented_df2['localidad_por_API'] == 'NO HUBO REQUEST'].shape[0]/augmented_df2.shape[0] * 100

7.868241001350083

Aproximadamente un 7.8% no pudo ser identificado.<br>
Tenemos que mencionar que confiar en el codigo postal lejos esta de ser el mejor de los casos (los sujetos pueden olvidarlo o ingresarlo erroneamente sin darse cuenta) pero frente los inconvenientes debidos a la falta de normalizacion de la base inicial nos parecio la mejor opcion. 
Hay que aclarar que otro problema es que los usuarios de CABA usualmente no ponen la localidad en el campo `loc_part` por lo que deberiamos hacer un sistema doble en el cual se asigne el campo de `provincia_por_cp` a `provincia_por_API`

---

In [74]:
augmented_df2[augmented_df2.localidad_por_API == 'CABALLITO'].sample(10) 
#  Debo arreglar que todos los que dice CABA y no tengan localidad por CP pasen a ser solo Capital

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
35167,2016,AR,CF,CABA,1437,ID_35167,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
43383,2017,AR,CF,CABA,1284,ID_43383,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
76265,2019,AR,CF,CABA,1406,ID_76265,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
33760,2016,AR,CF,CABA,1406,ID_33760,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
18707,2015,AR,CF,CABA,1429,ID_18707,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
50526,2018,AR,CF,CABA,1405,ID_50526,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
46388,2017,AR,CF,CABA,1406,ID_46388,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
76729,2019,AR,CF,CABA,1405,ID_76729,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856
28252,2016,AR,CF,CABA,1181,ID_28252,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,CABALLITO,NaN,NO HUBO REQUEST,None,COMUNA 6,2042,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.6168,-58.4436,NaN,NaN,-34.616843,-58.443568,-34.614493,-58.445856


In [112]:
mask = augmented_df2[(augmented_df2.loc_part == 'CABA')  &
              (augmented_df2.localidad_por_cp == 'NO UNICA LOCALIDAD') &
              (augmented_df2.provincia_por_cp == 'CIUDAD AUTONOMA DE BUENOS AIRES')].index

In [86]:
cols_modif = [9,10,11,12,13,14,17,18,19,20,21,22]
augmented_df2.iloc[mask,cols_modif] = 'NO HUBO REQUEST'

In [118]:
#  Debo arreglar que todos los que dicen `CDA AUTONOMA DE BS AS` o tengan localidad por CP pasen a ser solo Capital
mask = augmented_df2[(augmented_df2.loc_part == 'CDAD AUTONOMA BS AS') &
              (augmented_df2.localidad_por_cp == 'NO UNICA LOCALIDAD') &
              (augmented_df2.provincia_por_cp == 'CIUDAD AUTONOMA DE BUENOS AIRES')].index

augmented_df2.loc[mask,'provincia_por_API'] = 'CIUDAD AUTÓNOMA DE BUENOS AIRES'
augmented_df2.loc[mask,'provincia_id_API'] = 2
augmented_df2.loc[mask,'centroide_lat_provincia'] = -34.614493
augmented_df2.loc[mask,'centroide_lon_provincia'] = -58.445856

In [119]:
augmented_df2

,anio_ing,nacionalidad,provincia_part,loc_part,cp_part,user_id,provincia_por_cp,localidad_por_cp,departamento_por_cp,localidad_por_API,localidad_id_API,municipio_por_API,municipio_id_API,departamento_por_API,departamento_id_API,provincia_por_API,provincia_id_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_municipio,centroide_lon_municipio,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,
0,2014,AR,CF,CABA,1416,ID_0,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
1,2014,AR,CF,CABA,1416,ID_1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
2,2014,AR,BA,HURLINGHAM,1686,ID_2,BUENOS AIRES,HURLINGHAM,None,HURLINGHAM,06408010001,HURLINGHAM,60408,HURLINGHAM,6408,BUENOS AIRES,6,-34.5931,-58.6357,-34.5992,-58.6497,-34.5992,-58.6497,-36.676942,-60.558832
3,2014,AR,BA,ZARATE,2800,ID_3,BUENOS AIRES,NO UNICA LOCALIDAD,None,ZARATE,NaN,ZÁRATE,60882,ZÁRATE,6882,BUENOS AIRES,6,-34.0999,-59.0246,-33.9971,-59.1285,-33.9971,-59.1285,-36.676942,-60.558832
4,2014,AR,CF,CABA,1425,ID_4,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,-34.614493,-58.445856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87397,2019,AR,CF,CDAD AUTONOMA BS AS,1417,ID_87397,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NaN,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,-34.614493,-58.445856
87398,2019,AR,CF,CDAD AUTONOMA BS AS,1182,ID_87398,CIUDAD AUTONOMA DE BUENOS AIRES,PALERMO,COMUNA 14,PALERMO,02098010001,NO HUBO REQUEST,NaN,COMUNA 14,2098,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,-34.5739,-58.4224,NaN,NaN,-34.5738,-58.4222,-34.614493,-58.445856
87399,2019,AR,CF,CDAD AUTONOMA BS AS,1440,ID_87399,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,NO UNICO DEPARTAMENTO,NO HUBO REQUEST,NaN,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,CIUDAD AUTÓNOMA DE BUENOS AIRES,2,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,-34.614493,-58.445856


In [140]:
print(augmented_df2[augmented_df2.provincia_por_API == 'NO HUBO REQUEST'].shape[0] / augmented_df2.shape[0] * 100)
print(augmented_df2[augmented_df2.localidad_por_API == 'NO HUBO REQUEST'].shape[0] / augmented_df2.shape[0] * 100)
print(augmented_df2[augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES'].shape[0] / augmented_df2.shape[0] * 100)

print(augmented_df2[(augmented_df2.localidad_por_API == 'NO HUBO REQUEST') &
                    (augmented_df2.provincia_por_API != 'CIUDAD AUTÓNOMA DE BUENOS AIRES')].shape[0] / augmented_df2.shape[0] * 100)

print(augmented_df2[(augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES') & (augmented_df2.localidad_id_API != 'NO HUBO REQUEST')].shape[0] /
      augmented_df2[augmented_df2.provincia_por_API == 'CIUDAD AUTÓNOMA DE BUENOS AIRES'].shape[0] * 100)

4.104025079517632
44.384567858859064
45.168302784833294
4.104025079517632
19.154972389685394


Posterior a las modificaciones solo el 4% del padron no tiene un campo para provincia. <br>
Un 44.3 % no tiene una localidad definida. De ese 44% la mayoria responde a personas de CABA (un 45.1 % del padron). Si excluimos a esta gente solo un 4.1 % del padron no tiene una localidad normalizada. <br>
Por otro lado, los usuarios de CABA que SI POSEEN localidad definida son solo el 19.1 % del padron de CABA


---

In [128]:
pickle.dump(augmented_df2, open('HI_localizacion_socios.pickle','wb'))